# pandasのtips

In [18]:
import pandas as pd
import pandas_profiling as pdp
from sklearn import datasets

## テキストファイルの読み込み

* 適切な処理を行ったファイルを普通に読み込む場合は、pd.read_csv()で問題ない

* shift-jisになっていたり、機種依存文字が含まれていたりすると、read_csv()では読み込めないことがあるので、注意が必要

In [3]:
# 適切な処理を行ったファイルの読み込み
df_1 = pd.read_csv("./data/pandas_tips_testdata_1.txt", sep='\t')
df_1.head()

,id,user_id,name,create_time,update_time
0,1,111111111,あああ,2018-05-01 00:00:00,2018-05-01 00:00:00
1,2,222222222,いいい,2018-05-01 00:00:00,2018-05-01 00:00:00
2,3,333333333,ううう,2018-05-01 00:00:00,2018-05-01 00:00:00
3,4,444444444,えええ,2018-05-01 00:00:00,2018-05-01 00:00:00
4,5,555555555,おおお,2018-05-01 00:00:00,2018-05-01 00:00:00


## shift-jisで機種依存文字が入っているファイルの読み込み

shift-jisのファイルは、読み込み時にエンコードを指定すれば読み込める場合が多い

機種依存文字があると、エンコードを指定しても読み込めない

このような場合はopen関数でerrors="ignore"を指定して、機種依存文字を読み込んでから、pandasに渡せばよい

組み込みのopenでも、codecs.openでも読み込み可

この際、機種依存文字は削除された状態で読み込まれる


## nullが多いファイルの読み込み

上記の機種依存文字があるようなファイルを読み込む際、右端のカラムにnullが存在すると、DataFrameに渡す際にカラム数が合わないというエラーが出る

この場合は、カラム名のリストを渡すことで、カラム数を明示して読み込むことができる


## 巨大なファイルの読み込み

また、巨大なファイルを扱う場合にデータが疎なカラムが存在すると、最初に推測したデータ型が後から出てきたデータに合わずにエラーが出ることがある

この際は、読み込み時にdtypeを指定しておくことで、エラーを回避できる

In [12]:
# 機種依存文字があるshift-jisファイルの読み込み
with open("./data/pandas_tips_testdata_2.txt", mode="r", encoding="Shift-JIS", errors="ignore") as file:
    # カラムのリストを作成
    col_names = file.readline()
    col_list = col_names.replace("\r", "")\
                    .replace("\n", "")\
                    .split("\t")
    # DataFrameに変換
    df_2 = pd.read_table(file
                         , delimiter="\t" # 区切り文字の指定
                         , names=col_list # カラム名の明示
                         , dtype={"id": int, "create_time": str} # データ型の指定
                        )

df_2.head()

,id,user_id,name,create_time,update_time
0,1,111111111,あああ,2018-05-01 00:00:00,2018-05-01 00:00:00
1,2,222222222,いいい,2018-05-01 00:00:00,2018-05-01 00:00:00
2,3,333333333,ううう,2018-05-01 00:00:00,2018-05-01 00:00:00
3,4,444444444,えええ,2018-05-01 00:00:00,2018-05-01 00:00:00
4,5,555555555,おおお,2018-05-01 00:00:00,2018-05-01 00:00:00


## データの確認

pandasにはデータを確認するメソッドがいくつか用意されている

* DataFrame.info()

* DataFrame.describe()

* pandas-profiling  
┗https://qiita.com/h_kobayashi1125/items/02039e57a656abe8c48f



In [20]:
# irisのデータを使用
iris = datasets.load_iris()
df_iris = pd.DataFrame(iris.data, columns=iris.feature_names)
df_iris["target"] = iris.target_names[iris.target]
df_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [21]:
df_iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
sepal length (cm)    150 non-null float64
sepal width (cm)     150 non-null float64
petal length (cm)    150 non-null float64
petal width (cm)     150 non-null float64
target               150 non-null object
dtypes: float64(4), object(1)
memory usage: 5.9+ KB


In [22]:
df_iris.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [23]:
pdp.ProfileReport(df_iris)

Number of variables,5
Number of observations,150
Total Missing (%),0.0%
Total size in memory,5.9 KiB
Average record size in memory,40.5 B
Numeric,3
Categorical,1
Boolean,0
Date,0
Text (Unique),0
Rejected,1


## データハンドリングあれこれ

In [26]:
# str型の時系列データをdatetime型に変換
df_2["create_time"] = pd.to_datetime(df_2["create_time"])
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
id             5 non-null int32
user_id        5 non-null int64
name           5 non-null object
create_time    5 non-null datetime64[ns]
update_time    5 non-null object
dtypes: datetime64[ns](1), int32(1), int64(1), object(2)
memory usage: 260.0+ bytes


In [27]:
# select distinct的な
df_2["create_time"].unique()

array(['2018-05-01T00:00:00.000000000'], dtype='datetime64[ns]')